# Cleaning data in Python

This notebook will walk you through the process of reading in raw data from an outside source (in this case, stored on Github), cleaning, and exporting the final data set.

I will be working with two data sets: bi-weekly COVID case counts and median income data from the 2019 American Community Survey, both reported at the census tract level in King County, Washington. 

As is common for working with data tables in Python, I will be using pandas to store data in a dataframe for easy reference and manipulation.

In [87]:
import pandas as pd

## Step 1: Reading in data

As previously mentioned, the raw data is currently stored in a Github repository. Using the pandas functions .read_excel() and .read_csv() (for data in Excel and csv formats, respectively), you simply must provide a link to the raw data.

In [88]:
home = 'https://github.com/Group8-GovAnalyticsProject/GraceYoung/'

#Reading COVID data

covid_link = home + 'raw/main/biweekly-counts-rates-by-geography-dec-29.xlsx'
covid_cases = pd.read_excel(covid_link, sheet_name='Census')

In [90]:
#Reading median income data

income_link = home + 'raw/main/median_income_raw.csv'
income = pd.read_csv(link_income)

At this point, we have two dataframes - one storing raw COVID case data, and the other with raw median income information.

## Step 2: Preprocessing

In this step, we are prepping the data for cleaning. For the COVID case data, there are many records for each census tract because the data is reported bi-weekly. For merging purposes later, we want a single record for each census tract. Using the groupby() function, we summed the number of positive test results by census tract ID over two different time periods. First, based on the date at which the park data was collected, we summed all COVID cases that occured prior to that date. We also separately summed up data from the entire reporting time period (through January 2021).

The preprocessing for median income is much more straightforward. In this case, we simply selected the columns relevant to our work using .iloc.

In [89]:
# Preprocessing COVID data

# Filter for weeks prior to park counts collected
covid_cases_pre = covid_cases[covid_cases['Week_Start'] <= '2020-04-27'] #This is the date when park data was collected.

# Summing up the number of COVID cases per tract prior to the date where park counts were collected
covid_cases_pre = covid_cases_pre.groupby(['Census'], as_index=False)['Positives'].sum()
covid_cases_pre.columns = ['tract_id','precount_positive']

# Also summing all cases to date because having a lot of records with 0 is not good for later regression analysis
covid_cases_all = covid_cases.groupby(['Census'], as_index=False)['Positives'].sum()
covid_cases_all.columns = ['tract_id','all_positive']

In [91]:
# Preprocessing median income data

#Selecting only the columns I want (census tract ID and median household income)
income = income.iloc[:,[0,24]]
income.head()

,GEO_ID,S1901_C01_012E
0,id,Estimate!!Households!!Median income (dollars)
1,1400000US53033000100,55143
2,1400000US53033000200,77463
3,1400000US53033000300,100917
4,1400000US53033000401,32877


At this point, we now have three dataframes: (1) COVID cases prior to park visit data collection by census tract, (2) all-time COVID cases by census tract, (3) median income by census tract.

The first two dataframes are nicely cleaned and ready for merging. Median income, however, needs a bit more work.

## Step 3: Cleaning

Looking at the income dataframe, two problems are immediately apparent. First, the first row of data contains further description of the column names, and the actual data does not start until the second row. Moreso, the second column does not easily describe what data is represented in that column. Second, GEO_ID is a long character string, in which the census tract ID is contained in the final 9 characters. In order to be able to merge on this column, we must extract just these digits. Finally, there are some non-numeric income records but we want income to be strictly numeric.

In [92]:
#Dropping the first text row

income.drop(0, axis=0, inplace=True)
income.reset_index(drop=True, inplace=True)
income.head()

,GEO_ID,S1901_C01_012E
0,1400000US53033000100,55143
1,1400000US53033000200,77463
2,1400000US53033000300,100917
3,1400000US53033000401,32877
4,1400000US53033000402,72150


In [93]:
#Renaming columns
income.columns = ['tract_id', 'median_hhold_inc']
income.head()

,tract_id,median_hhold_inc
0,1400000US53033000100,55143
1,1400000US53033000200,77463
2,1400000US53033000300,100917
3,1400000US53033000401,32877
4,1400000US53033000402,72150


In [94]:
#Fixing geo_id to just show tract ID by dropping the first 9 characters
drop_first = lambda s : int(str(s)[9:])

income['tract_id'] = income['tract_id'].apply(drop_first)
income.head()

,tract_id,median_hhold_inc
0,53033000100,55143
1,53033000200,77463
2,53033000300,100917
3,53033000401,32877
4,53033000402,72150


In [95]:
#Fixing column data types
income['median_hhold_inc'] = pd.to_numeric(income['median_hhold_inc'], errors='coerce')

#Removing any NaNs
income = income.dropna()

At this point, all three dataframes are cleaned (standard column names, shared census tract ID for later merging, appropriate column data types) and ready for the merging step.

## Step 4: Exporting data for later use

The .to_csv() function saves dataframes to the current working directory in a csv format.

In [96]:
#Writing final cleaned csv files
covid_cases_pre.to_csv('clean_covid_cases_pre.csv', index=False)
covid_cases_all.to_csv('clean_covid_cases_all.csv', index=False)
income.to_csv('clean_median_income.csv', index=False)